# ADS Capstone Project: Airbnb Pricing Prediction
#### By Renetta Nelson, Michael Nguyen and Jacqueline Urenda

# Modeling

In [13]:
#importing libraries

import pandas as pd
import numpy as np
import random
import xgboost
from xgboost import XGBRegressor


from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score
from tabulate import tabulate

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

import warnings
warnings.filterwarnings('ignore')

In [2]:
%store -r X_train
%store -r X_test
%store -r y_train
%store -r y_test


## Decision Tree Modeling

#### Setting baseline decision tree model

In [3]:
#fitting baseline regressor

DCTR = DecisionTreeRegressor()
DCTR.fit(X_train,y_train)


DecisionTreeRegressor()

#### Evaluating performance of baseline model

In [4]:
y_pred = DCTR.predict(X_test)

In [5]:
y_pred_train = DCTR.predict(X_train)

In [6]:
MSE = mean_squared_error(y_test, y_pred)
RMSE = mean_squared_error(y_test, y_pred, squared = False)
R2 = r2_score(y_test, y_pred)
MAPE = mean_absolute_percentage_error(y_test, y_pred)


n = len(y_test)
k = len(X_test.columns)
Adjusted_R2 = 1-(((1-R2)*(n-1))/(n-k-1))

In [7]:
DCTR_Results = [['MSE', 'RMSE', 'R2', 'Adjusted_R2', 'MAPE'], 
         [MSE, RMSE, R2, Adjusted_R2, MAPE]]

print(tabulate(DCTR_Results))

-----------------  ------------------  ------------------  ------------------  ------------------
MSE                RMSE                R2                  Adjusted_R2         MAPE
66934.93773981836  258.71787286505423  0.9620149641003602  0.9620139857349796  0.1828629788219595
-----------------  ------------------  ------------------  ------------------  ------------------


## XGBoost Rgression Model

###

In [10]:
import xgboost
from xgboost import XGBRegressor

In [13]:
XGBR = XGBRegressor()

XGBR.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=100, n_jobs=None, num_parallel_tree=None,
             predictor=None, random_state=None, ...)

In [14]:
y_pred_XGB = XGBR.predict(X_test)

In [17]:
XGB_MSE = mean_squared_error(y_test, y_pred_XGB)
XGB_RMSE = mean_squared_error(y_test, y_pred_XGB, squared = False)
XGB_R2 = r2_score(y_test, y_pred_XGB)
XGB_MAPE = mean_absolute_percentage_error(y_test, y_pred_XGB)


n = len(y_test)
k = len(X_test.columns)
XGB_Adjusted_R2 = 1-(((1-R2)*(n-1))/(n-k-1))

In [18]:
XGB_Results = [['MSE', 'RMSE', 'R2', 'Adjusted_R2', 'MAPE'], 
         [XGB_MSE, XGB_RMSE,XGB_R2, XGB_Adjusted_R2, XGB_MAPE]]

print(tabulate(XGB_Results))

----------------  -----------------  ------------------  ------------------  ------------------
MSE               RMSE               R2                  Adjusted_R2         MAPE
72983.4207401773  270.1544386830935  0.9585824989085637  0.9620139857349796  0.3090160273277967
----------------  -----------------  ------------------  ------------------  ------------------


## Random Forrest Model

In [3]:
#convert 
sample_indices = np.random.choice(X_train.shape[0], 5000, replace=False)
X_train_sample = X_train.iloc[sample_indices, :]
y_train_sample = y_train.iloc[sample_indices]

#### Fitting baseline model:

In [4]:
#fit baseline model
baseline_rf = RandomForestRegressor(random_state=42)
baseline_rf.fit(X_train_sample, y_train_sample)

RandomForestRegressor(random_state=42)

In [5]:
#importance
importances = baseline_rf.feature_importances_

In [6]:
#sort by importance(10)
indices = np.argsort(importances)[-10:]
X_train_reduced = X_train.iloc[:, indices]

In [7]:
top_features = X_train.columns[indices]

# Select top features from the sample
X_train_sample_reduced = X_train_sample[top_features]

#### Setting up hyperparemeters

In [8]:
#set up hyperparameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
max_features = ['auto', 'sqrt']
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth}

In [9]:
rf = RandomForestRegressor(random_state=42)

In [10]:
rf_random = RandomizedSearchCV(estimator=rf, param_distributions=random_grid, n_iter=100, cv=3, verbose=2, random_state=42, n_jobs=-1)

In [14]:
#fit random search
rf_random.fit(X_train_sample_reduced, y_train_sample)

Fitting 3 folds for each of 100 candidates, totalling 300 fits


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviou

[CV] END .max_depth=10, max_features=auto, n_estimators=1400; total time=  27.5s


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV] END .max_depth=20, max_features=auto, n_estimators=1000; total time=  32.0s
[CV] END .max_depth=20, max_features=auto, n_estimators=1000; total time=  32.0s


...

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV] END ..max_depth=90, max_features=auto, n_estimators=600; total time=  22.5s


/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


[CV] END ..max_depth=90, max_features=auto, n_estimators=600; total time=  22.9s
[CV] END .max_depth=80, max_features=auto, n_estimators=1400; total time=  46.9s
[CV] END .max_depth=80, max_features=auto, n_estimators=1400; total time=  43.4s
[CV] END .max_depth=80, max_features=auto, n_estimators=1400; total time=  41.8s


RandomizedSearchCV(cv=3, estimator=RandomForestRegressor(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'n_estimators': [200, 400, 600, 800,
                                                         1000, 1200, 1400, 1600,
                                                         1800, 2000]},
                   random_state=42, verbose=2)

In [15]:
best_params = rf_random.best_params_
print("Best parameters: ", best_params)

Best parameters:  {'n_estimators': 400, 'max_features': 'sqrt', 'max_depth': 60}


In [16]:
rf_best = RandomForestRegressor(n_estimators=best_params['n_estimators'], max_depth=best_params['max_depth'], max_features=best_params['max_features'], random_state=42)
rf_best.fit(X_train_reduced, y_train)

RandomForestRegressor(max_depth=60, max_features='sqrt', n_estimators=400,
                      random_state=42)

In [17]:
# Reduce test set
X_test_reduced = X_test[top_features]

# Predict on test set
y_pred_rf = rf_best.predict(X_test_reduced)

In [18]:
#SHAP and interpret
importances = rf_best.feature_importances_
feature_importance_df = pd.DataFrame({"Feature": top_features, "Importance": importances})
feature_importance_df = feature_importance_df.sort_values(by="Importance", ascending=False)
print(feature_importance_df)

                  Feature  Importance
5              listing_id    0.393552
8  host_neighbourhood_cat    0.244225
9          minimum_nights    0.192621
6            accommodates    0.052072
0     review_scores_value    0.035999
2       number_of_reviews    0.026642
4                bedrooms    0.017599
7                bathroom    0.016849
3                    beds    0.016261
1            average_temp    0.004182


In [20]:
rf_MSE = mean_squared_error(y_test, y_pred_rf)
rf_RMSE = mean_squared_error(y_test, y_pred_rf, squared = False)
rf_R2 = r2_score(y_test, y_pred_rf)
rf_MAPE = mean_absolute_percentage_error(y_test, y_pred_rf)


n = len(y_test)
k = len(X_test_reduced.columns)
rf_Adjusted_R2 = 1-(((1-rf_R2)*(n-1))/(n-k-1))

In [22]:
RF_Results = [['MSE', 'RMSE', 'R2', 'Adjusted_R2', 'MAPE'], 
         [rf_MSE, rf_RMSE,rf_R2, rf_Adjusted_R2, rf_MAPE]]

print(tabulate(RF_Results))

-----------------  -----------------  ------------------  ------------------  -------------------
MSE                RMSE               R2                  Adjusted_R2         MAPE
77696.99503085097  278.7418071098252  0.9559075836148075  0.9559071293535326  0.16821962791261985
-----------------  -----------------  ------------------  ------------------  -------------------
